# AI-Powered Trading Bot Using Supervised Learning

### 1. Data Fetching from Binance API

In [ ]:
# AI-Powered Trading Bot Using Supervised Learning

## 1. Install Dependencies

!pip install python-binance pandas scikit-learn ccxt


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.6/118.6 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.9/69.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 80.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 67.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.1/164.1 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.7/288.7 kB 24.9 MB/s eta 0:00:00


In [ ]:
# Import necessary libraries
import pandas as pd
from binance.client import Client
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import os


In [ ]:
# 2. Data Fetching from Binance API

def fetch_binance_data(api_key, api_secret, symbol, interval='1d', start_str='1 year ago UTC'):
    client = Client(api_key, api_secret)
    klines = client.get_historical_klines(symbol, interval, start_str)

    # Convert the fetched data into a DataFrame
    data = pd.DataFrame(klines, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume',
                                         'close_time', 'quote_asset_volume', 'number_of_trades',
                                         'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
    # Converting to appropriate data types
    data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms')
    data['open'] = data['open'].astype(float)
    data['high'] = data['high'].astype(float)
    data['low'] = data['low'].astype(float)
    data['close'] = data['close'].astype(float)
    data['volume'] = data['volume'].astype(float)

    return data


### 2. Feature Engineering

In [ ]:
# 3. Feature Engineering

def create_features(data):
    data['ma50'] = data['close'].rolling(window=50).mean()
    data['ma200'] = data['close'].rolling(window=200).mean()
    data['rsi'] = compute_rsi(data['close'], 14)

    # Filling missing values with the closest non-NA values
    data.fillna(method='ffill', inplace=True)
    data.dropna(inplace=True)

    return data

def compute_rsi(series, period=14):
    delta = series.diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi


### 3. Model Training (Random Forest)

In [ ]:
# 4. Model Training (Random Forest)

def train_model(data):
    # Define features and target
    X = data[['ma50', 'ma200', 'rsi']]
    # Target: 1 if next day's close is higher than current close, else 0
    data['target'] = (data['close'].shift(-1) > data['close']).astype(int)
    y = data['target']

    # Split data into train/test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    # Train a RandomForest Classifier
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # Predict and evaluate the model
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f'Model Accuracy: {accuracy * 100:.2f}%')

    return model


### 4. Backtesting

In [ ]:
# 5. Backtesting

def backtest_strategy(data, model):
    data['predicted_signal'] = model.predict(data[['ma50', 'ma200', 'rsi']])

    # Buy if the model predicts an upward movement, sell otherwise
    data['position'] = data['predicted_signal'].shift(1)

    # Calculate daily returns
    data['returns'] = data['close'].pct_change()

    # Calculate strategy returns based on the position (buy or sell)
    data['strategy_returns'] = data['returns'] * data['position']

    # Calculate cumulative returns
    data['cumulative_returns'] = (1 + data['strategy_returns']).cumprod() - 1

    print(f"Cumulative Returns: {data['cumulative_returns'].iloc[-1] * 100:.2f}%")
    return data


### 5. Main Script

In [ ]:


API_KEY = os.getenv('****')
API_SECRET = os.getenv('****')
SYMBOL = 'BTCUSDT'

# Step 1: Fetch Data
data = fetch_binance_data(API_KEY, API_SECRET, SYMBOL)

# Step 2: Feature Engineering
data = create_features(data)

# Step 3: Train the model
model = train_model(data)

# Step 4: Backtest the strategy
backtest_result = backtest_strategy(data, model)

# Show the backtesting results
backtest_result[['timestamp', 'close', 'cumulative_returns']].tail(10)


<ipython-input-15-3613eb5eaf23>:9: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill', inplace=True)


Model Accuracy: 55.88%
Cumulative Returns: 297.25%


,timestamp,close,cumulative_returns
356,2024-09-20,63201.05,2.972508
357,2024-09-21,63348.96,2.972508
358,2024-09-22,63578.76,2.972508
359,2024-09-23,63339.99,2.972508
360,2024-09-24,64262.70,2.972508
361,2024-09-25,63152.01,2.972508
362,2024-09-26,65173.99,2.972508
363,2024-09-27,65769.95,2.972508
364,2024-09-28,65858.00,2.972508
365,2024-09-29,65761.74,2.972508
